In [ ]:
# Imports a utilizar 
import numpy as np
import matplotlib as plt
import pandas as pd
import seaborn as sns
import plotly.express as px

from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC

### Abrir la data y observar su tipo de variable

In [ ]:
# Crea una lista vacía para guardar los DataFrames
dfs = []

for year in range(2009, 2019):
    filename = f"data{year}.sav"
    df = pd.read_spss(filename)
    dfs.append(df)
    df['Year'] = year

# Concatena los DataFrames en uno solo
merged_df = pd.concat(dfs)

# Muestra las primeras 5 filas del DataFrame combinado
print(merged_df['Year'].unique())

In [ ]:
merged_df.columns

In [ ]:
merged_df.dtypes

In [ ]:
columns = merged_df.columns
numeric_data = merged_df._get_numeric_data().columns

categorical_data = set(columns) - set(numeric_data)
print("-> Columnas con valores numericos \n ",numeric_data ,'\n')
print("-> Columnas con valores categoricos\n ",categorical_data)

### Limpieza de Variables

Se toman las columnas que van a ser utilizadas en el modelo predictivo. Estas son:
- Departamento de registro
- Municipio de registro
- Escolaridad de la madre
- Escolaridad del padre
- Año de registro

In [ ]:
columnas_deseadas = ['Depreg', 'Escolam', 'Escolap', 'Year', 'Asisrec', 'Sitioocu']
data = merged_df.drop(columns=merged_df.columns.difference(columnas_deseadas))

Vamos a verificar si la data que estamos tomando contiene valores nulos o NaN.

Podemos observar que en nuestro dataset a utilizar contamos con algunos registros faltantes. Estos estan ubicados especificamente en las columnas de escolaridad de la madre y del padre del nacido.

In [ ]:
data.columns

In [ ]:
data

In [ ]:
data.shape

## Análisis Exploratorio

Podemos observar que en nuestro data set contamos con una unica variable numerica siendo el año del registro. Luego en cuanto a la escolaridad de ambos padres y el municipio y departamento de registro lo encontramos como categorico.

In [ ]:
data['Escolam'] = data['Escolam'].fillna('Ninguno')
data['Escolap'] = data['Escolap'].fillna('Ninguno')

In [ ]:
sns.heatmap(data.isna(), cmap='viridis')
plt.show()

In [ ]:
data['Escolam'].unique()

In [ ]:
data['Escolap'] = data['Escolap'].replace(['Post Grado', 'Post grado'], 'Postgrado')
data['Escolam'] = data['Escolap'].replace(['Post Grado', 'Post grado'], 'Postgrado')
data['Escolap'] = data['Escolap'].replace('Básico', 'Básica')
data['Escolam'] = data['Escolap'].replace('Básico', 'Básica')


In [ ]:
data['Escolam'].unique()

In [ ]:
data['Escolap'].unique()

Ahora vamos a codificar la escolaridad de la madre y del padre asignandoles un valor numerico a dichas variables. Para ello vamos a usar el diccionario de variables directamente brindado por el Instituto Nacional de Estadistica donde asignan:
- 1: Ninguno
- 2: Primaria
- 3: Básica
- 4: Diversificado
- 5: Universitario
- 6: Postgrado
- 7: Ignorado
- 8: Doctorado

In [ ]:
data['Escolam'] = data['Escolam'].replace({
    'Ninguno': 1,
    'Primaria': 2,
    'Básica': 3,
    'Diversificado': 4,
    'Universitario': 5,
    'Postgrado': 6,
    'Ignorado': 7,
    'Doctorado': 8
})
data['Escolap'] = data['Escolap'].replace({
    'Ninguno': 1,
    'Primaria': 2,
    'Básica': 3,
    'Diversificado': 4,
    'Universitario': 5,
    'Postgrado': 6,
    'Ignorado': 7,
    'Doctorado': 8
})


Realizaremos el mismo proceso para codificar el municipio y departamento del registro de nacimiento. Utilizando el diccionario brindado oficial obtenemos:
Departamentos
1. Guatemala
2. El Progreso
3. Sacatepéquez
4. Chimaltenango
5. Escuintla
6. Santa Rosa
7. Sololá
8. Totonicapán
9. Quetzaltenango
10. Suchitepéquez
11. Retalhuleu
12. San Marcos
13. Huehuetenango
14. Quiché
15. Baja Verapaz
16. Alta Verapaz
17. Petén
18. Izabal
19. Zacapa
20. Chiquimula
21. Jalapa
22. Jutiapa

In [ ]:
data['Depreg'] = data['Depreg'].replace({
    'Guatemala': 1,
    'El Progreso': 2,
    'Sacatepéquez': 3,
    'Chimaltenango': 4,
    'Escuintla': 5,
    'Santa Rosa': 6,
    'Sololá': 7,
    'Solola': 7,
    'Totonicapán': 8,
    'Quetzaltenango': 9,
    'Suchitepéquez': 10,
    'Retalhuleu': 11,
    'San Marcos': 12,
    'Huehuetenango': 13,
    'Quiché': 14,
    'Baja Verapaz': 15,
    'Alta Verapaz': 16,
    'Petén': 17,
    'Izabal': 18,
    'Zacapa': 19,
    'Chiquimula': 20,
    'Jalapa': 21,
    'Jutiapa': 22,
    'Guatemala': 1,
    'El Progreso': 2,
    'Sacatepequez': 3,
    'Chimaltenango': 4,
    'Escuintla': 5,
    'Santa Rosa': 6,
    'Solola': 7,
    'Totonicapan': 8,
    'Quetzaltenango': 9,
    'Suchitepecquez': 10,
    'Retalhuleu': 11,
    'San Marcos': 12,
    'Huehuetenango': 13,
    'Quiche': 14,
    'Baja Verapaz': 15,
    'Alta Verapaz': 16,
    'Peten': 17,
    'Izabal': 18,
    'Zacapa': 19,
    'Chiquimula': 20,
    'Jalapa': 21,
    'Jutiapa': 22,
    'Suchitepequez':10
})


In [ ]:
print(list(data['Depreg'].unique()))

Ahora codificamos la asistencia recibida y el sitio de ocurrencia <br>
Asistencia
- 'Medico':1
- 'Personal de Enfermeria':2
- 'Paramedico':3
- 'Comadrona':4
- 'Empirico':5
- 'Ninguno':6
- 'Ignorado':7

Sitio ocurrencia
- 'Hospital público':1
- 'Hospital privado':2
- 'Centro de salud':3
- 'Seguro social':4
- 'Vìa pública':5
- 'Domicilio':6
- 'Otro':7
- 'Ignorado':8

In [ ]:
data['Asisrec'] = data['Asisrec'].replace({
'Medico':1,
'Medica':1,
'Médica':1,
'Personal de Enfermeria':2,
'Paramedico':3,
'Paramédica':3,
'Comadrona':4,
'Empirico':5,
'Empirica':5,
'Empírica':5,
'Ninguno':6,
'Ninguna':6,
'Ignorado':7
})

data['Sitioocu'] = data['Sitioocu'].replace({
'Hospital público':1,
'Hospital':1,
'Hospital privado':2,
'Centro de salud':3,
'Casa de salud':3,
'Seguro social':4,
'Vìa pública':5,
'Via pública':5,
'Via publica':5,
'Vía Pública':5,
'Domicilio':6,
'Otro':7,
'Ignorado':8

})

In [ ]:
data['Sitioocu'].unique()

In [ ]:
data['Asisrec'].unique()

In [ ]:
data.dtypes

In [ ]:
TESTLIMIT = 1000
data_model = data.sample(TESTLIMIT)
X = data_model[['Depreg',	'Sitioocu',	'Year',	'Escolap', 'Escolam']].values
y = data_model.iloc[:, 1].values
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# Agrupar los datos por 'Year' y 'Asisrec' y contar la frecuencia de cada combinación
grouped_data = data_model.groupby(['Year', 'Asisrec']).size().reset_index(name='Count')

# Ordenar los datos por 'Year'
grouped_data = grouped_data.sort_values(by='Year')

# Graficar los resultados utilizando un gráfico de línea o de barras
plt.figure(figsize=(10, 10))
sns.lineplot(x='Year', y='Count', hue='Asisrec', data=grouped_data)
# O bien, utilizar un gráfico de barras agrupado
sns.barplot(x='Year', y='Count', hue='Asisrec', data=grouped_data, ci=None)
plt.xlabel('Año')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de Asistencia a lo largo de los años')
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

model = SVC(kernel = 'linear', random_state = 0)
model.fit(X_entreno, y_entreno)

y_pred = model.predict(X_prueba)

np.concatenate((y_pred.reshape(len(y_pred),1), y_pred.reshape(len(y_pred),1)),1)
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_prueba, y_pred))

cm = confusion_matrix(y_prueba, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
x = data.filter(["Sitioocu", "Asisrec"]) # TODO cambiar a variables a utilizar

kmeans = KMeans(3)
kmeans.fit(x)

clusters_identificados = kmeans.fit_predict(x)

# Revisar los resultados

clusters_identificados

datos_con_clusters = data.copy()

# Crear una nueva serie, que tenga el cluster identificado para cada observación

datos_con_clusters['Cluster'] = clusters_identificados

# Revisar el resultado

datos_con_clusters

datos_con_clusters["Cluster"] = datos_con_clusters["Cluster"].astype("category")

fig = px.scatter(
    datos_con_clusters, 
    x = "Sitioocu", 
    y = "Asisrec", 
    color = "Cluster",
#    size = '', 
    hover_data = ['Depreg']
)
fig.update_xaxes(range=[-180, 180])
fig.update_yaxes(range=[-90, 90])

fig.show()

Ya con nuestro dataset codificado podemos proseguir